In [1]:
# !pip install overpy --quiet

In [2]:
import overpy
import pandas as pd
import folium
import numpy as np
from geopy.distance import geodesic
api = overpy.Overpass()

In [3]:
df = pd.read_csv("new_cluster_data.csv")
df.head()

,lattitude,longitude,demand
0,8.543299,76.974782,505.0
1,8.553152,76.973958,422.0
2,8.543449,76.987987,706.0
3,8.553915,76.959806,743.0
4,8.501149,76.956882,1009.0


In [4]:
lat_lon = df[["lattitude","longitude"]]
geo_data = lat_lon.to_numpy()

In [5]:
geo_data

array([[ 8.54329936, 76.97478213],
       [ 8.55315226, 76.97395804],
       [ 8.54344948, 76.98798658],
       [ 8.55391519, 76.9598055 ],
       [ 8.50114881, 76.95688193],
       [ 8.49350885, 76.93163997],
       [ 8.48297355, 76.94275079],
       [ 8.49094277, 76.95057625],
       [ 8.50527068, 76.94854734],
       [ 8.47224587, 76.96973312],
       [ 8.48369405, 76.96906918],
       [ 8.49366999, 76.96132246],
       [ 8.49159223, 76.98074672],
       [ 8.47962662, 76.95862286],
       [ 8.54561859, 76.90645879],
       [ 8.55101259, 76.91745979],
       [ 8.56055427, 76.90336774],
       [ 8.50464919, 76.98715724],
       [ 8.52965966, 76.99083198],
       [ 8.51149662, 77.00089014],
       [ 8.52707501, 77.00257778],
       [ 8.50961065, 76.97387795],
       [ 8.52233472, 76.98216635],
       [ 8.44148386, 76.94767209],
       [ 8.45047668, 76.9481868 ],
       [ 8.44957008, 76.94059541],
       [ 8.52210699, 76.92895439],
       [ 8.52631197, 76.91454361],
       [ 8.52812993,

In [6]:
api = overpy.Overpass()
array_of_closest_road_coordinates = []

for coordinates in geo_data:
    lat, lon = coordinates
    geo_coordinates = []

    # Define your query to fetch roads within a certain distance from the provided coordinates
    result = api.query(f"""
        way(around:200, {lat}, {lon}) ["highway"];
        (._;>;);
        out body;
    """)

    for way in result.ways:
        geo_coordinates.append([(node.lat, node.lon) for node in way.nodes])

    # Initialize variables to store the closest road and its distance
    closest_road = None
    min_distance = float('inf')  # Initialize with infinity

    # Iterate through each road and calculate the distance
    for road in geo_coordinates:
        for coord in road:
            # coord is a tuple (latitude, longitude)
            road_location = (float(coord[0]), float(coord[1]))

            # Calculate the distance between the input location and the road
            distance = geodesic(coordinates, road_location).meters

            # Update the closest road if a shorter distance is found
            if distance < min_distance:
                min_distance = distance
                closest_road = road_location

    array_of_closest_road_coordinates.append(closest_road)


In [7]:
locations = np.array(array_of_closest_road_coordinates)

In [8]:
length = len(locations)

In [9]:
mean_latitude = np.mean(locations[:, 0])
mean_longitude = np.mean(locations[:, 1])

In [17]:
m = folium.Map([mean_latitude, mean_longitude], zoom_start=12)
for i in range(len(geo_data)):
  folium.Marker(
      location=geo_data[i],
      popup=geo_data[i],
      icon=folium.Icon(color="red"),
  ).add_to(m)
m

In [11]:
m = folium.Map([mean_latitude, mean_longitude], zoom_start=12)
for i in range(length):
  folium.Marker(
      location=locations[i],
      popup=locations[i],
      icon=folium.Icon(color="green"),
  ).add_to(m)
m

In [12]:
locations

array([[ 8.5433879, 76.9748978],
       [ 8.5525729, 76.9736185],
       [ 8.5432623, 76.9879534],
       [ 8.5538579, 76.9597945],
       [ 8.5010363, 76.9567119],
       [ 8.4936063, 76.9314593],
       [ 8.482737 , 76.9428993],
       [ 8.4911899, 76.9505725],
       [ 8.5054925, 76.9485801],
       [ 8.4719637, 76.9697575],
       [ 8.4838878, 76.9690481],
       [ 8.4935142, 76.9610109],
       [ 8.4915059, 76.9806963],
       [ 8.479622 , 76.9585   ],
       [ 8.5455722, 76.9064955],
       [ 8.5514003, 76.9178611],
       [ 8.5605588, 76.9036923],
       [ 8.5046789, 76.9871612],
       [ 8.5294864, 76.9910891],
       [ 8.5114364, 77.0009493],
       [ 8.527456 , 77.0026967],
       [ 8.5094562, 76.9740299],
       [ 8.5223243, 76.9822042],
       [ 8.441428 , 76.9475999],
       [ 8.4502968, 76.9484015],
       [ 8.4493394, 76.9407376],
       [ 8.5220618, 76.929166 ],
       [ 8.5263403, 76.9145989],
       [ 8.5280786, 76.9406355],
       [ 8.5164455, 76.9044898],
       [ 8

In [13]:
np.shape(locations)

(37, 2)

In [14]:
df["lat"] = locations[:,0]
df["lon"] = locations[:,1]

In [15]:
df.head()

,lattitude,longitude,demand,lat,lon
0,8.543299,76.974782,505.0,8.543388,76.974898
1,8.553152,76.973958,422.0,8.552573,76.973619
2,8.543449,76.987987,706.0,8.543262,76.987953
3,8.553915,76.959806,743.0,8.553858,76.959795
4,8.501149,76.956882,1009.0,8.501036,76.956712


In [16]:
coord = df[["lat","lon","demand"]]
coord.to_csv("new_collection_points.csv",index=False)